In [1]:
PATH='/home/kirana/Documents/phd/exp3_autoencoder'

In [2]:
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/bookcrossing'

In [3]:
import pandas as pd
import numpy as np
import os
import datetime
import sklearn
import torch

In [4]:
ls {DATAPATH}/*.csv

/home/kirana/Documents/final_dissertation_final/experiments/datasets/bookcrossing/BX-Book-Ratings.csv
/home/kirana/Documents/final_dissertation_final/experiments/datasets/bookcrossing/BX-Books.csv
/home/kirana/Documents/final_dissertation_final/experiments/datasets/bookcrossing/BX-Users.csv


## Read Files

In [5]:
ratings=pd.read_csv(f'{DATAPATH}/BX-Book-Ratings.csv',encoding='latin-1',error_bad_lines=False,sep=";")
print (ratings.head())


   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6


In [6]:
ratings.columns=['userId','itemId','rating']

In [7]:
items=pd.read_csv (f'{DATAPATH}/BX-Books.csv',encoding='latin-1',error_bad_lines=False,sep=";")
print(items.head())


b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/

/home/kirana/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
items.rename({'ISBN':'itemId'},axis=1,inplace=True)

In [9]:
items.head()

,itemId,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [10]:
users=pd.read_csv(f'{DATAPATH}/BX-Users.csv',encoding='latin-1',error_bad_lines=False,sep=";")
print (users.head())

   User-ID                            Location   Age
0        1                  nyc, new york, usa   NaN
1        2           stockton, california, usa  18.0
2        3     moscow, yukon territory, russia   NaN
3        4           porto, v.n.gaia, portugal  17.0
4        5  farnborough, hants, united kingdom   NaN


In [11]:
users.rename({'User-ID':'userId'},axis=1,inplace=True)

In [12]:
users.head()

,userId,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [13]:
ratings.shape

(1149780, 3)

In [14]:
ratings=pd.merge(ratings,users,on='userId',how='left')

In [15]:
ratings.shape

(1149780, 5)

In [16]:
items.columns

Index(['itemId', 'Book-Title', 'Book-Author', 'Year-Of-Publication',
       'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [17]:
ratings=pd.merge(ratings,items.iloc[:,:5],on='itemId',how='left')

In [18]:
ratings.shape

(1149780, 9)

In [19]:
ratings.head()

,userId,itemId,rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,034545104X,0,"tyler, texas, usa",NaN,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,276726,0155061224,5,"seattle, washington, usa",NaN,Rites of Passage,Judith Rae,2001,Heinle
2,276727,0446520802,0,"h, new south wales, australia",16.0,The Notebook,Nicholas Sparks,1996,Warner Books
3,276729,052165615X,3,"rijeka, n/a, croatia",16.0,Help!: Level 1,Philip Prowse,1999,Cambridge University Press
4,276729,0521795028,6,"rijeka, n/a, croatia",16.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press


## Random Cross-Validation 90-10 split to compare to papers

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
ratings['rating'].value_counts()

0     716109
8     103736
10     78610
7      76457
9      67541
5      50974
6      36924
4       8904
3       5996
2       2759
1       1770
Name: rating, dtype: int64

In [22]:
temptrain,tempvalid=train_test_split(ratings,test_size=0.1,train_size=0.9,random_state=11,shuffle=True)

In [23]:
temptrain['random_dstype']='train'
tempvalid['random_dstype']='test'

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
df=pd.concat([temptrain,tempvalid],axis=0)

In [25]:
df.shape

(1149780, 10)

In [26]:
df.head()

,userId,itemId,rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,random_dstype
1006309,241656,0340824735,7,"melbourn, england, united kingdom",49.0,NaN,NaN,NaN,NaN,train
608046,147045,0873512553,0,"saint paul, minnesota, usa",28.0,The days of Rondo,Evelyn Fairbanks,1990,Minnesota Historical Society Press,train
871538,210926,0345459202,9,"guelph, ontario, canada",NaN,Big Stone Gap,Adriana Trigiani,2003,Fawcett Books,train
752379,182085,0446350451,0,"ginsheim, hessen, germany",NaN,Sensations,Virginia Carry,1988,Warner Books,train
124597,28591,0821767445,10,"michie, tennessee, usa",33.0,Jed (Rock Creek Six),Linda Devlin,2001,Zebra Books,train


## Prepare data

In [27]:
user_to_idx={j:i+1 for i, j in enumerate(df['userId'].unique())}
item_to_idx={j:i+1 for i,j in enumerate(df['itemId'].unique())}
idx_to_user={i+1:j for i, j in enumerate(df['userId'].unique())}
idx_to_item={i+1:j for i,j in enumerate(df['itemId'].unique())}

In [28]:
df['user_idx']=[user_to_idx.get(i) for i in df['userId']]
df['item_idx']=[item_to_idx.get(i) for i in df['itemId']]

In [29]:
df['dstype_random_train']=np.where(df['random_dstype']=='train',1,0)

In [30]:
df['dstype_random_valid']=np.where(df['random_dstype']!='train',1,0)

In [31]:
df.shape

(1149780, 14)

In [32]:
from scipy.sparse import csr_matrix

In [34]:
from scipy.sparse import csr_matrix

def df_to_sm(data, vars_i, vars_j, var_val):
    grpr_i = data.groupby(vars_i).grouper

    idx_i = grpr_i.group_info[0]

    grpr_j = data.groupby(vars_j).grouper

    idx_j = grpr_j.group_info[0]

    data_sm = csr_matrix((data[var_val].values, (idx_i, idx_j)),
                         shape=(grpr_i.ngroups+1, grpr_j.ngroups+1))

    return data_sm, grpr_i, grpr_j



In [35]:
df.head()

,userId,itemId,rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,random_dstype,user_idx,item_idx,dstype_random_train,dstype_random_valid
1006309,241656,0340824735,7,"melbourn, england, united kingdom",49.0,NaN,NaN,NaN,NaN,train,1,1,1,0
608046,147045,0873512553,0,"saint paul, minnesota, usa",28.0,The days of Rondo,Evelyn Fairbanks,1990,Minnesota Historical Society Press,train,2,2,1,0
871538,210926,0345459202,9,"guelph, ontario, canada",NaN,Big Stone Gap,Adriana Trigiani,2003,Fawcett Books,train,3,3,1,0
752379,182085,0446350451,0,"ginsheim, hessen, germany",NaN,Sensations,Virginia Carry,1988,Warner Books,train,4,4,1,0
124597,28591,0821767445,10,"michie, tennessee, usa",33.0,Jed (Rock Creek Six),Linda Devlin,2001,Zebra Books,train,5,5,1,0


In [36]:
df_ratings,_,_=df_to_sm(df,['user_idx'],['item_idx'],'rating')

In [37]:
type(df_ratings), df_ratings.shape

(scipy.sparse.csr.csr_matrix, (105284, 340557))

In [38]:
pd.__version__

'0.24.2'

In [39]:
df_ratings.shape

(105284, 340557)

In [40]:
dfflagtrain,_,_=df_to_sm(df,['user_idx'],['item_idx'],'dstype_random_train')


In [41]:
dfflagvalid,_,_=df_to_sm(df,['user_idx'],['item_idx'],'dstype_random_valid')



In [42]:
df_ratings.shape,dfflagtrain.shape,dfflagvalid.shape

((105284, 340557), (105284, 340557), (105284, 340557))

In [43]:
df_ratings.shape,dfflagtrain.shape

((105284, 340557), (105284, 340557))

In [44]:
df_train=df_ratings.multiply(dfflagtrain)

In [45]:
df_valid=df_ratings.multiply(dfflagvalid)

In [46]:
import pickle

In [47]:
pickle.dump([df,df_train,df_valid,df,df_ratings,idx_to_user,\
             idx_to_item,item_to_idx,user_to_idx],open(f'{DATAPATH}/reads.pkl','wb'))

In [48]:
len(user_to_idx), len(item_to_idx)

(105283, 340556)

In [49]:
df_ratings,_,_=df_to_sm(df,['item_idx'],['user_idx'],'rating')

In [50]:
type(df_ratings), df_ratings.shape

(scipy.sparse.csr.csr_matrix, (340557, 105284))

In [51]:
dfflagtrain,_,_=df_to_sm(df,['item_idx'],['user_idx'],'dstype_random_train')


In [52]:
dfflagvalid,_,_=df_to_sm(df,['item_idx'],['user_idx'],'dstype_random_valid')



In [53]:
df_ratings.shape,dfflagtrain.shape,dfflagvalid.shape

((340557, 105284), (340557, 105284), (340557, 105284))

In [54]:
df_train=df_ratings.multiply(dfflagtrain)

In [55]:
df_valid=df_ratings.multiply(dfflagvalid)

In [56]:
pickle.dump([df,df_train,df_valid,df,df_ratings,idx_to_user,\
             idx_to_item,item_to_idx,user_to_idx],open(f'{DATAPATH}/itemreads.pkl','wb'))